<a href="https://colab.research.google.com/github/BlackeWhite/SSD_Modanet/blob/master/Prediction%26analysis_Instagram_Images/Prediction%26analysis_on_Instagram_Image_FFSSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [ ]:
#per salvare file di gorsse dimensioni sul drive 
!pip install httplib2==0.15.0
!pip install google-api-python-client==1.6
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install keras==2.2.3
!pip install tensorflow-gpu==1.15
import os.path
if not(os.path.isdir('keras_layers')):
  !git clone https://github.com/pierluigiferrari/ssd_keras.git
  !mv ./ssd_keras/* . #copy all file in the root directory, you must this because after invocate the lib in the repository 
  !rm -r ssd_keras

from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import SGD
from imageio import imread
import numpy as np
from matplotlib import pyplot as plt

from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib
plt.ioff()


# Import Instagram Images and weight of FFSSD

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Immagini Instagram estratte dagli hastag
Test = drive.CreateFile({'id': '1lloX-DOgQ7bG7iT0a0HMpfMNhOa-UvYi'}) 
Test.GetContentFile(Test['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(Test['title'], Test['id']))
!unzip instagram_image.zip
#pesi FFSSD
Weight = drive.CreateFile({'id': '1-5PCZq_LDfzCkFGS6W0Kre6Q9se4nyey'}) 
Weight.GetContentFile(Weight['title'])  # Save Drive file as a local file
print('Uploaded {}, id {}'.format(Weight['title'], Weight['id']))

# Definition FFSSD and load model

In [ ]:
"""Keras implementation of FSSD."""
from __future__ import division
import numpy as np
from keras.models import Model
from keras.layers import Input, Lambda, Activation, Conv2D, MaxPooling2D, ZeroPadding2D, Reshape, Concatenate,Deconvolution2D, add
from keras.regularizers import l2
import keras.backend as K

from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_L2Normalization import L2Normalization
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast


def FSSD(image_size,
            n_classes,
            mode='training',
            l2_regularization=0.0005,
            min_scale=None,
            max_scale=None,
            scales=None,
            aspect_ratios_global=None,
            aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                     [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                     [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                     [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                     [1.0, 2.0, 0.5],
                                     [1.0, 2.0, 0.5]],
            two_boxes_for_ar1=True,
            steps=[8, 16, 32, 64, 100, 300],
            offsets=None,
            clip_boxes=False,
            variances=[0.1, 0.1, 0.2, 0.2],
            coords='centroids',
            normalize_coords=True,
            subtract_mean=[123, 117, 104],
            divide_by_stddev=None,
            swap_channels=[2, 1, 0],
            confidence_thresh=0.01,
            iou_threshold=0.45,
            top_k=200,
            nms_max_output_size=400,
            return_predictor_sizes=False):
    """SSD300 architecture.
    # Arguments
        input_shape: Shape of the input image,
            expected to be either (300, 300, 3) or (3, 300, 300)(not tested).
        num_classes: Number of classes including background.
    # References
        https://arxiv.org/abs/1512.02325
    """

    n_predictor_layers = 6 # The number of predictor conv layers in the network is 6 for the original SSD300.
    n_classes += 1 # Account for the background class.
    l2_reg = l2_regularization # Make the internal name shorter.
    img_height, img_width, img_channels = image_size[0], image_size[1], image_size[2]

    ############################################################################
    # Get a few exceptions out of the way.
    ############################################################################

    if aspect_ratios_global is None and aspect_ratios_per_layer is None:
        raise ValueError("`aspect_ratios_global` and `aspect_ratios_per_layer` cannot both be None. At least one needs to be specified.")
    if aspect_ratios_per_layer:
        if len(aspect_ratios_per_layer) != n_predictor_layers:
            raise ValueError("It must be either aspect_ratios_per_layer is None or len(aspect_ratios_per_layer) == {}, but len(aspect_ratios_per_layer) == {}.".format(n_predictor_layers, len(aspect_ratios_per_layer)))

    if (min_scale is None or max_scale is None) and scales is None:
        raise ValueError("Either `min_scale` and `max_scale` or `scales` need to be specified.")
    if scales:
        if len(scales) != n_predictor_layers+1:
            raise ValueError("It must be either scales is None or len(scales) == {}, but len(scales) == {}.".format(n_predictor_layers+1, len(scales)))
    else: # If no explicit list of scaling factors was passed, compute the list of scaling factors from `min_scale` and `max_scale`
        scales = np.linspace(min_scale, max_scale, n_predictor_layers+1)

    if len(variances) != 4:
        raise ValueError("4 variance values must be pased, but {} values were received.".format(len(variances)))
    variances = np.array(variances)
    if np.any(variances <= 0):
        raise ValueError("All variances must be >0, but the variances given are {}".format(variances))

    if (not (steps is None)) and (len(steps) != n_predictor_layers):
        raise ValueError("You must provide at least one step value per predictor layer.")

    if (not (offsets is None)) and (len(offsets) != n_predictor_layers):
        raise ValueError("You must provide at least one offset value per predictor layer.")

    ############################################################################
    # Compute the anchor box parameters.
    ############################################################################

    # Set the aspect ratios for each predictor layer. These are only needed for the anchor box layers.
    if aspect_ratios_per_layer:
        aspect_ratios = aspect_ratios_per_layer
    else:
        aspect_ratios = [aspect_ratios_global] * n_predictor_layers

    # Compute the number of boxes to be predicted per cell for each predictor layer.
    # We need this so that we know how many channels the predictor layers need to have.
    if aspect_ratios_per_layer:
        n_boxes = []
        for ar in aspect_ratios_per_layer:
            if (1 in ar) & two_boxes_for_ar1:
                n_boxes.append(len(ar) + 1) # +1 for the second box for aspect ratio 1
            else:
                n_boxes.append(len(ar))
    else: # If only a global aspect ratio list was passed, then the number of boxes is the same for each predictor layer
        if (1 in aspect_ratios_global) & two_boxes_for_ar1:
            n_boxes = len(aspect_ratios_global) + 1
        else:
            n_boxes = len(aspect_ratios_global)
        n_boxes = [n_boxes] * n_predictor_layers

    if steps is None:
        steps = [None] * n_predictor_layers
    if offsets is None:
        offsets = [None] * n_predictor_layers

    ############################################################################
    # Define functions for the Lambda layers below.
    ############################################################################

    def identity_layer(tensor):
        return tensor

    def input_mean_normalization(tensor):
        return tensor - np.array(subtract_mean)

    def input_stddev_normalization(tensor):
        return tensor / np.array(divide_by_stddev)

    def input_channel_swap(tensor):
        if len(swap_channels) == 3:
            return K.stack([tensor[...,swap_channels[0]], tensor[...,swap_channels[1]], tensor[...,swap_channels[2]]], axis=-1)
        elif len(swap_channels) == 4:
            return K.stack([tensor[...,swap_channels[0]], tensor[...,swap_channels[1]], tensor[...,swap_channels[2]], tensor[...,swap_channels[3]]], axis=-1)

    ############################################################################
    # Build the network.
    ############################################################################

    x = Input(shape=(img_height, img_width, img_channels))

    # The following identity layer is only needed so that the subsequent lambda layers can be optional.
    x1 = Lambda(identity_layer, output_shape=(img_height, img_width, img_channels), name='identity_layer')(x)
    if not (subtract_mean is None):
        x1 = Lambda(input_mean_normalization, output_shape=(img_height, img_width, img_channels), name='input_mean_normalization')(x1)
    if not (divide_by_stddev is None):
        x1 = Lambda(input_stddev_normalization, output_shape=(img_height, img_width, img_channels), name='input_stddev_normalization')(x1)
    if swap_channels:
        x1 = Lambda(input_channel_swap, output_shape=(img_height, img_width, img_channels), name='input_channel_swap')(x1)

    # Block 1
    conv1_1 = Conv2D(64, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv1_1')(x1)
    conv1_2 = Conv2D(64, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv1_2')(conv1_1)
    pool1 = MaxPooling2D((2, 2), strides=(2, 2), padding='same',name='pool1')(conv1_2)
    # Block 2
    conv2_1 = Conv2D(128, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv2_1')(pool1)
    conv2_2 = Conv2D(128, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv2_2')(conv2_1)
    pool2 = MaxPooling2D((2, 2), strides=(2, 2), padding='same',name='pool2')(conv2_2)
    # Block 3
    conv3_1 = Conv2D(256, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv3_1')(pool2)
    conv3_2 = Conv2D(256, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv3_2')(conv3_1)
    conv3_3 = Conv2D(256, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv3_3')(conv3_2)
    pool3 = MaxPooling2D((2, 2), strides=(2, 2), padding='same',name='pool3')(conv3_3)
    # Block 4
    conv4_1 = Conv2D(512, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv4_1')(pool3)
    conv4_2 = Conv2D(512, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv4_2')(conv4_1)
    conv4_3 = Conv2D(512, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv4_3')(conv4_2)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2), padding='same',name='pool4')(conv4_3)
    # Block 5
    conv5_1 = Conv2D(512, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv5_1')(pool4)
    conv5_2 = Conv2D(512, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv5_2')(conv5_1)
    conv5_3 = Conv2D(512, (3, 3),activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv5_3')(conv5_2)
    pool5 = MaxPooling2D((3, 3), strides=(1, 1), padding='same',name='pool5')(conv5_3)
    # FC6
    fc6 = Conv2D(1024, (3, 3), dilation_rate=(6, 6),activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='fc6')(pool5)
    #fc6 = Dropout(0.5, name='drop6')(fc6)
    # FC7
    fc7 = Conv2D(1024, (1, 1), padding='same', activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='fc7')(fc6)
    #fc7 = Dropout(0.5, name='drop7')(fc7)
    # Block 6
    conv6_1 = Conv2D(256, (1, 1), activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv6_1')(fc7)
    conv6_2 = Conv2D(512, (3, 3), strides=(2, 2),activation='relu', padding='same',name='conv6_2')(conv6_1)
    # Block 7
    conv7_1 = Conv2D(128, (1, 1), activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv7_1')(conv6_2)
    conv7_2 = ZeroPadding2D()(conv7_1)
    conv7_2 = Conv2D(256, (3, 3), strides=(2, 2),activation='relu', padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv7_2')(conv7_2)
    # Block 8
    conv8_1 = Conv2D(128, (1, 1), activation='relu',padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv8_1')(conv7_2)
    conv8_2 = Conv2D(256, (3, 3), strides=(2, 2),activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv8_2')(conv8_1)
    # Last Pool
    conv9_1 = Conv2D(128, (1, 1), activation='relu', padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv9_1')(conv8_2)
    pool6 = Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='pool6')(conv9_1)
    
    # Prediction from conv4_3
    conv4=Conv2D(512,(3,3),padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg),name='conv4')(conv4_3)
    conv4_3_norm = L2Normalization(gamma_init=20, name='conv4_3_norm')(conv4)
    deconv5_3 = Deconvolution2D(512, (3, 3),strides=(2, 2),padding='same',name='deconv5_3')(conv5_3)
    conv_deconv5_3=Conv2D(512, (3, 3),padding='same',name='conv_deconv5_3')(deconv5_3)
    conv5_3_norm = L2Normalization(gamma_init=20,name='conv_5_3_norm')(conv_deconv5_3)
    EltSUM=add([conv4_3_norm,conv5_3_norm])
    fusion_conv4_5=Activation(name='fusion_conv4_5',activation='relu')(EltSUM)
    
    ### Build the convolutional predictor layers on top of the base network

    # We precidt `n_classes` confidence values for each box, hence the confidence predictors have depth `n_boxes * n_classes`
    # Output shape of the confidence layers: `(batch, height, width, n_boxes * n_classes)`
    conv4_5_norm_mbox_conf = Conv2D(n_boxes[0] * n_classes, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv4_5_norm_mbox_conf')(fusion_conv4_5)
    fc7_mbox_conf = Conv2D(n_boxes[1] * n_classes, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='fc7_mbox_conf')(fc7)
    conv6_2_mbox_conf = Conv2D(n_boxes[2] * n_classes, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv6_2_mbox_conf')(conv6_2)
    conv7_2_mbox_conf = Conv2D(n_boxes[3] * n_classes, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv7_2_mbox_conf')(conv7_2)
    conv8_2_mbox_conf = Conv2D(n_boxes[4] * n_classes, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv8_2_mbox_conf')(conv8_2)
    pool6_mbox_conf = Conv2D(n_boxes[5] * n_classes, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='pool6_mbox_conf')(pool6)
    # We predict 4 box coordinates for each box, hence the localization predictors have depth `n_boxes * 4`
    # Output shape of the localization layers: `(batch, height, width, n_boxes * 4)`
    conv4_5_norm_mbox_loc = Conv2D(n_boxes[0] * 4, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv4_5_norm_mbox_loc')(fusion_conv4_5)
    fc7_mbox_loc = Conv2D(n_boxes[1] * 4, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='fc7_mbox_loc')(fc7)
    conv6_2_mbox_loc = Conv2D(n_boxes[2] * 4, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv6_2_mbox_loc')(conv6_2)
    conv7_2_mbox_loc = Conv2D(n_boxes[3] * 4, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv7_2_mbox_loc')(conv7_2)
    conv8_2_mbox_loc = Conv2D(n_boxes[4] * 4, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='conv8_2_mbox_loc')(conv8_2)
    pool6_mbox_loc = Conv2D(n_boxes[5] * 4, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(l2_reg), name='pool6_mbox_loc')(pool6)
    
    #Generate priors conv4_5_norm_mbox_priorbox
    conv4_5_norm_mbox_priorbox = AnchorBoxes(img_height, img_width, this_scale=scales[0], next_scale=scales[1], aspect_ratios=aspect_ratios[0],
                                             two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[0], this_offsets=offsets[0], clip_boxes=clip_boxes,
                                             variances=variances, coords=coords, normalize_coords=normalize_coords, name='conv4_5_norm_mbox_priorbox')(conv4_5_norm_mbox_loc)
    #Generate priors fc7
    fc7_mbox_priorbox = AnchorBoxes(img_height, img_width, this_scale=scales[1], next_scale=scales[2], aspect_ratios=aspect_ratios[1],
                                    two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[1], this_offsets=offsets[1], clip_boxes=clip_boxes,
                                    variances=variances, coords=coords, normalize_coords=normalize_coords, name='fc7_mbox_priorbox')(fc7_mbox_loc)
    #Generate priors conv6_2
    conv6_2_mbox_priorbox = AnchorBoxes(img_height, img_width, this_scale=scales[2], next_scale=scales[3], aspect_ratios=aspect_ratios[2],
                                        two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[2], this_offsets=offsets[2], clip_boxes=clip_boxes,
                                        variances=variances, coords=coords, normalize_coords=normalize_coords, name='conv6_2_mbox_priorbox')(conv6_2_mbox_loc)
    #Generate priors conv7_2
    conv7_2_mbox_priorbox = AnchorBoxes(img_height, img_width, this_scale=scales[3], next_scale=scales[4], aspect_ratios=aspect_ratios[3], 
                                        two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[3], this_offsets=offsets[3], clip_boxes=clip_boxes, 
                                        variances=variances, coords=coords, normalize_coords=normalize_coords, name='conv7_2_mbox_priorbox')(conv7_2_mbox_loc)
    #Generate priors conv8_2
    conv8_2_mbox_priorbox = AnchorBoxes(img_height, img_width, this_scale=scales[4], next_scale=scales[5], aspect_ratios=aspect_ratios[4],
                                        two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[4], this_offsets=offsets[4], clip_boxes=clip_boxes,
                                        variances=variances, coords=coords, normalize_coords=normalize_coords, name='conv8_2_mbox_priorbox')(conv8_2_mbox_loc)
    pool6_mbox_priorbox = AnchorBoxes(img_height, img_width, this_scale=scales[5], next_scale=scales[6], aspect_ratios=aspect_ratios[5],
                                        two_boxes_for_ar1=two_boxes_for_ar1, this_steps=steps[5], this_offsets=offsets[5], clip_boxes=clip_boxes,
                                        variances=variances, coords=coords, normalize_coords=normalize_coords, name='pool6_mbox_priorbox')(pool6_mbox_loc)
    
       ### Reshape

    # Reshape the class predictions, yielding 3D tensors of shape `(batch, height * width * n_boxes, n_classes)`
    # We want the classes isolated in the last axis to perform softmax on them
    conv4_5_norm_mbox_conf_reshape = Reshape((-1, n_classes), name='conv4_5_norm_mbox_conf_reshape')(conv4_5_norm_mbox_conf)
    fc7_mbox_conf_reshape = Reshape((-1, n_classes), name='fc7_mbox_conf_reshape')(fc7_mbox_conf)
    conv6_2_mbox_conf_reshape = Reshape((-1, n_classes), name='conv6_2_mbox_conf_reshape')(conv6_2_mbox_conf)
    conv7_2_mbox_conf_reshape = Reshape((-1, n_classes), name='conv7_2_mbox_conf_reshape')(conv7_2_mbox_conf)
    conv8_2_mbox_conf_reshape = Reshape((-1, n_classes), name='conv8_2_mbox_conf_reshape')(conv8_2_mbox_conf)
    pool6_mbox_conf_reshape = Reshape((-1, n_classes), name='pool6_mbox_conf_reshape')(pool6_mbox_conf)
    # Reshape the box predictions, yielding 3D tensors of shape `(batch, height * width * n_boxes, 4)`
    # We want the four box coordinates isolated in the last axis to compute the smooth L1 loss
    conv4_5_norm_mbox_loc_reshape = Reshape((-1, 4), name='conv4_3_norm_mbox_loc_reshape')(conv4_5_norm_mbox_loc)
    fc7_mbox_loc_reshape = Reshape((-1, 4), name='fc7_mbox_loc_reshape')(fc7_mbox_loc)
    conv6_2_mbox_loc_reshape = Reshape((-1, 4), name='conv6_2_mbox_loc_reshape')(conv6_2_mbox_loc)
    conv7_2_mbox_loc_reshape = Reshape((-1, 4), name='conv7_2_mbox_loc_reshape')(conv7_2_mbox_loc)
    conv8_2_mbox_loc_reshape = Reshape((-1, 4), name='conv8_2_mbox_loc_reshape')(conv8_2_mbox_loc)
    pool6_mbox_loc_reshape = Reshape((-1, 4), name='pool6_mbox_loc_reshape')(pool6_mbox_loc)
    # Reshape the anchor box tensors, yielding 3D tensors of shape `(batch, height * width * n_boxes, 8)`
    conv4_5_norm_mbox_priorbox_reshape = Reshape((-1, 8), name='conv4_5_norm_mbox_priorbox_reshape')(conv4_5_norm_mbox_priorbox)
    fc7_mbox_priorbox_reshape = Reshape((-1, 8), name='fc7_mbox_priorbox_reshape')(fc7_mbox_priorbox)
    conv6_2_mbox_priorbox_reshape = Reshape((-1, 8), name='conv6_2_mbox_priorbox_reshape')(conv6_2_mbox_priorbox)
    conv7_2_mbox_priorbox_reshape = Reshape((-1, 8), name='conv7_2_mbox_priorbox_reshape')(conv7_2_mbox_priorbox)
    conv8_2_mbox_priorbox_reshape = Reshape((-1, 8), name='conv8_2_mbox_priorbox_reshape')(conv8_2_mbox_priorbox)
    pool6_mbox_priorbox_reshape = Reshape((-1, 8), name='pool6_mbox_priorbox_reshape')(pool6_mbox_priorbox)
    
    # Gather all predictions
    mbox_conf = Concatenate(axis=1, name='mbox_conf')([conv4_5_norm_mbox_conf_reshape,
                                                       fc7_mbox_conf_reshape,
                                                       conv6_2_mbox_conf_reshape,
                                                       conv7_2_mbox_conf_reshape,
                                                       conv8_2_mbox_conf_reshape,
                                                       pool6_mbox_conf_reshape])
    
    mbox_loc = Concatenate(axis=1, name='mbox_loc')([conv4_5_norm_mbox_loc_reshape,
                                                     fc7_mbox_loc_reshape,
                                                     conv6_2_mbox_loc_reshape,
                                                     conv7_2_mbox_loc_reshape,
                                                     conv8_2_mbox_loc_reshape,
                                                     pool6_mbox_loc_reshape])
    
    mbox_priorbox = Concatenate(axis=1, name='mbox_priorbox')([conv4_5_norm_mbox_priorbox_reshape,
                                                               fc7_mbox_priorbox_reshape,
                                                               conv6_2_mbox_priorbox_reshape,
                                                               conv7_2_mbox_priorbox_reshape,
                                                               conv8_2_mbox_priorbox_reshape,
                                                               pool6_mbox_priorbox_reshape])
    mbox_conf_softmax = Activation('softmax', name='mbox_conf_softmax')(mbox_conf)
    predictions = Concatenate(axis=2, name='predictions')([mbox_conf_softmax, mbox_loc, mbox_priorbox])
    if mode == 'training':
        model = Model(inputs=x, outputs=predictions)
    elif mode == 'inference':
        decoded_predictions = DecodeDetections(confidence_thresh=confidence_thresh,
                                               iou_threshold=iou_threshold,
                                               top_k=top_k,
                                               nms_max_output_size=nms_max_output_size,
                                               coords=coords,
                                               normalize_coords=normalize_coords,
                                               img_height=img_height,
                                               img_width=img_width,
                                               name='decoded_predictions')(predictions)
        model = Model(inputs=x, outputs=decoded_predictions)
    elif mode == 'inference_fast':
        decoded_predictions = DecodeDetectionsFast(confidence_thresh=confidence_thresh,
                                                   iou_threshold=iou_threshold,
                                                   top_k=top_k,
                                                   nms_max_output_size=nms_max_output_size,
                                                   coords=coords,
                                                   normalize_coords=normalize_coords,
                                                   img_height=img_height,
                                                   img_width=img_width,
                                                   name='decoded_predictions')(predictions)
        model = Model(inputs=x, outputs=decoded_predictions)
    else:
        raise ValueError("`mode` must be one of 'training', 'inference' or 'inference_fast', but received '{}'.".format(mode))

    if return_predictor_sizes:
        predictor_sizes = np.array([conv4_5_norm_mbox_conf._keras_shape[1:3],
                                     fc7_mbox_conf._keras_shape[1:3],
                                     conv6_2_mbox_conf._keras_shape[1:3],
                                     conv7_2_mbox_conf._keras_shape[1:3],
                                     conv8_2_mbox_conf._keras_shape[1:3],
                                     pool6_mbox_conf._keras_shape[1:3]])
        return model, predictor_sizes
    else:
        return model

In [ ]:
img_height = 300 # Height of the model input images
img_width = 300 # Width of the model input images
img_channels = 3 # Number of color channels of the model input images
mean_color = [123, 117, 104] # The per-channel mean of the images in the dataset.
swap_channels = [2, 1, 0] # The color channel order in the original SSD is BGR
n_classes = 13
scales_coco = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05] # The anchor box scaling factors used in the original SSD300 for the MS COCO datasets
scales = scales_coco
aspect_ratios = [[1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                 [1.0, 2.0, 0.5],
                 [1.0, 2.0, 0.5]] # The anchor box aspect ratios used in the original SSD300; 
two_boxes_for_ar1 = True
steps = [8, 16, 32, 64, 100, 300] # The space between two adjacent anchor box center points for each predictor layer.
offsets = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5] # The offsets of the first anchor box center points from the top and left borders of the image as a fraction of the step size for each predictor layer.
clip_boxes = False # Whether or not to clip the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are divided as in the original implementation
normalize_coords = True
model_mode = 'inference'

In [ ]:
# 1: Build the Keras model

K.clear_session() # Clear previous models from memory.

model = FSSD(image_size=(img_height, img_width, img_channels),
                n_classes=n_classes,
                mode='inference',
                l2_regularization=0.0005,
                scales=scales,
                aspect_ratios_per_layer=aspect_ratios,
                two_boxes_for_ar1=two_boxes_for_ar1,
                steps=steps,
                offsets=offsets,
                clip_boxes=clip_boxes,
                variances=variances,
                normalize_coords=normalize_coords,
                subtract_mean=mean_color,
                swap_channels=swap_channels)


# 2: Load the trained weights into the model.

# TODO: Set the path of the trained weights.
weights_path = '/content/FFSSD_modanet_epoch-38_loss-5.1148_val_loss-5.8968.h5'

model.load_weights(weights_path, by_name=True)

# 3: Compile the model so that Keras won't complain the next time you load it.

sgd = SGD(lr=0.001, momentum=0.9, decay=0.0, nesterov=False)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=sgd, loss=ssd_loss.compute_loss, metrics=['acc'])

# Creation of folders where save predicted images

In [ ]:
!mkdir pred_image
!mkdir "pred_image/#tods"
!mkdir "pred_image/#fendi"
!mkdir "pred_image/#dolcegabbana"
!mkdir "pred_image/#puma"
!mkdir "pred_image/#valentino"

#  Prediction of Bounding box and analysis on Instagram Image

In [ ]:
import os

classes = ['background',
					 'bag', 'belt', 'boots', 'footwear',
					 'outer', 'dress', 'sunglasses', 'pants',
					 'top', 'shorts', 'skirt', 'headwear',
					 'scarf/tie']

lista_image = []
lista_hastag = []
lista_hastag = os.listdir("/content/instagram_image/")  #lista contenente i nomi delle 5 cartelle contenente le immagini dei diversi hast
for hast in lista_hastag:                              #viene effettuato per ogni hastag 
	lista_image= os.listdir("/content/instagram_image/" + hast)
	lista_bag=0               #contatori abiti
	lista_belt=0
	lista_boots=0
	lista_footwear=0
	lista_outer=0
	lista_dress=0
	lista_sunglasses=0
	lista_pants=0
	lista_top=0
	lista_shorts=0
	lista_skirt=0
	lista_headwear=0
	lista_scarfetie=0
	i=0 # conatore immagini predette 
	for item in lista_image:
		f = open(hast + ".csv", 'a')       #file csv per salvare le predizioni delle immagini predette
		wrt = open(hast + "_stats.txt", 'w')
		orig_images = [] # Store the images here.
		input_images = [] # Store resized versions of the images here.

		img_path = '/content/instagram_image/' + hast + "/" + item  

		orig_images.append(imread(img_path))
		img = image.load_img(img_path, target_size=(img_height, img_width))
		img = image.img_to_array(img) 
		input_images.append(img)
		input_images = np.array(input_images)
		y_pred = model.predict(input_images)
		confidence_threshold = 0.5
		y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]

		np.set_printoptions(precision=2, suppress=True, linewidth=90)
		print("Predicted boxes for ",item,"\n")
		print('class   conf  xmin   ymin   xmax   ymax')
		print(y_pred_thresh[0])
	  #stabiliamo una trashold basata sul numero di boundind box per ogni immagine predetta
	  #se il numero di boundind box è maggiore l'immagine viene processata e salvata 
	  #altrimenti viene scartata 

		if len(y_pred_thresh[0]) > 0:
			i=i+1
			f.write("Predicted boxes for image(300x300): " + str(item) + "\n")
			f.write('class,conf,xmin,ymin,xmax,ymax \n')
			for box in y_pred_thresh[0]:
				if round(box[0])==1:
					lista_bag+=1
				if round(box[0])==2:
					lista_belt+=1
				elif round(box[0])==3:
					lista_boots+=1
				elif round(box[0])==4:
					lista_footwear+=1 
				elif round(box[0])==5:
					lista_outer+=1                                                                            
				elif round(box[0])==6:
					lista_dress+=1
				elif round(box[0])==7:
					lista_sunglasses+=1   
				elif round(box[0])==8:
					lista_pants+=1  
				elif round(box[0])==9:
					lista_top+=1                                                          
				elif round(box[0])==10:
					lista_shorts+=1
				elif round(box[0])==11:
					lista_skirt+=1        
				elif round(box[0])==12:
					lista_headwear+=1   
				elif round(box[0])==13:
					lista_scarfetie+=1   
				f.write(str(box[0]) + "," + str(round(box[1],2)) + "," + str(round(box[2],2)) + "," + str(round(box[3],2)) + "," + str(round(box[4],2)) + "," + str(round(box[5],2)) +"\n" )
			colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
			plt.figure(figsize=(20,12))
			plt.axis('off')
			plt.imshow(orig_images[0])
			current_axis = plt.gca()
			f.write("\nPredicted boxes for original iamge: " + str(item) + "\n")
			f.write('class,conf,xmin,ymin,xmax,ymax \n')
			for box in y_pred_thresh[0]:
				# Transform the predicted bounding boxes for the 300x300 image to the original image dimensions.
				xmin = box[2] * orig_images[0].shape[1] / img_width
				ymin = box[3] * orig_images[0].shape[0] / img_height
				xmax = box[4] * orig_images[0].shape[1] / img_width
				ymax = box[5] * orig_images[0].shape[0] / img_height
				f.write(str(box[0]) + "," + str(round(box[1],2)) + "," + str(round(xmin,2)) + "," + str(round(ymin,2)) + "," + str(round(xmax,2)) + "," + str(round(ymax,2)) +"\n" )
				color = colors[int(box[0])]
				label = '{}: {:.2f}'.format(classes[int(box[0])], box[1])
				current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
				current_axis.text(xmin, ymin, label, size='x-large', color='black', bbox={'facecolor':color, 'alpha':1.0})  
				plt.savefig('./pred_image/' + hast + "/" + item, bbox_inches='tight', pad_inches=0) 
			plt.close()
			f.write("-------------------------------------- \n")
		del y_pred_thresh, y_pred
	wrt.write("N_images: " + str(i) + "\n")
	wrt.write("Bag: " + str(lista_bag)+ "\n")
	wrt.write("Belt: " + str(lista_belt)+ "\n")
	wrt.write("Boots: " + str(lista_boots)+ "\n")
	wrt.write("Footwear: " + str(lista_footwear)+ "\n")
	wrt.write("Outer: "+str(lista_outer)+ "\n")
	wrt.write("Dress: "+str(lista_dress)+ "\n")
	wrt.write("Sunglasses: "+str(lista_sunglasses)+ "\n")
	wrt.write("Pants: "+str(lista_pants)+ "\n")
	wrt.write("Top: "+str(lista_top)+ "\n")
	wrt.write("Shorts: "+str(lista_shorts)+ "\n")
	wrt.write("Skirt: "+str(lista_skirt)+ "\n")
	wrt.write("Headwear: "+str(lista_headwear)+ "\n")
	wrt.write("Scarf&tie: "+ str(lista_scarfetie)+ "\n")  
	wrt.close()
	f.close() 

# Save predict images and annotations

In [ ]:
!zip results.zip pred_image/*/* 
!cp results.zip "/content/drive/My Drive/FFSSD_maggiore_di_1(almeno due bounding box)"
!cp "#dolcegabbana_stats.txt" "/content/drive/My Drive/FFSSD_maggiore_di_1(almeno due bounding box)"
!cp "#valentino_stats.txt" "/content/drive/My Drive/FFSSD_maggiore_di_1(almeno due bounding box)"
!cp "#fendi_stats.txt" "/content/drive/My Drive/FFSSD_maggiore_di_1(almeno due bounding box)"
!cp "#puma_stats.txt" "/content/drive/My Drive/FFSSD_maggiore_di_1(almeno due bounding box)"
!cp "#tods_stats.txt" "/content/drive/My Drive/FFSSD_maggiore_di_1(almeno due bounding box)"
!cp "#dolcegabbana.csv" "/content/drive/My Drive/FFSSD_maggiore_di_1(almeno due bounding box)"
!cp "#valentino.csv" "/content/drive/My Drive/FFSSD_maggiore_di_1(almeno due bounding box)"
!cp "#fendi.csv" "/content/drive/My Drive/FFSSD_maggiore_di_1(almeno due bounding box)"
!cp "#puma.csv" "/content/drive/My Drive/FFSSD_maggiore_di_1(almeno due bounding box)"
!cp "#tods.csv" "/content/drive/My Drive/FFSSD_maggiore_di_1(almeno due bounding box)"

In [ ]:
!zip results.zip pred_image/*/* 
!cp results.zip "/content/drive/My Drive/FFSSD_maggiore_di_0(almeno un bounding box)"
!cp "#dolcegabbana_stats.txt" "/content/drive/My Drive/FFSSD_maggiore_di_0(almeno un bounding box)"
!cp "#valentino_stats.txt" "/content/drive/My Drive/FFSSD_maggiore_di_0(almeno un bounding box)"
!cp "#fendi_stats.txt" "/content/drive/My Drive/FFSSD_maggiore_di_0(almeno un bounding box)"
!cp "#puma_stats.txt" "/content/drive/My Drive/FFSSD_maggiore_di_0(almeno un bounding box)"
!cp "#tods_stats.txt" "/content/drive/My Drive/FFSSD_maggiore_di_0(almeno un bounding box)"
!cp "#dolcegabbana.csv" "/content/drive/My Drive/FFSSD_maggiore_di_0(almeno un bounding box)"
!cp "#valentino.csv" "/content/drive/My Drive/FFSSD_maggiore_di_0(almeno un bounding box)"
!cp "#fendi.csv" "/content/drive/My Drive/FFSSD_maggiore_di_0(almeno un bounding box)"
!cp "#puma.csv" "/content/drive/My Drive/FFSSD_maggiore_di_0(almeno un bounding box)"
!cp "#tods.csv" "/content/drive/My Drive/FFSSD_maggiore_di_0(almeno un bounding box)"